<a href="https://colab.research.google.com/github/Hyeok-Jun-Yoon/AI_Plus/blob/main/%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C_4%EC%A3%BC%EC%B0%A8_AI_2%EA%B8%B0_%EC%9C%A4%ED%98%81%EC%A4%80_key.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 심화과제: 수능 국어 문제 GPT-4로 풀어보기



In [1]:
!pip install openai datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


[MYCODE] API Key 설정

In [2]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('api_key'))

## [MYCODE] 수능 국어 문제 준비

In [ ]:
import pandas as pd
import requests

# GitHub URL에서 JSON 파일을 가져옵니다.
url = "https://raw.githubusercontent.com/NomaDamas/KICE_slayer_AI_Korean/refs/heads/master/data/2023_11_KICE.json"

# URL에서 JSON 데이터 가져오기
response = requests.get(url)

# 인코딩을 UTF-8로 설정하여 처리
response.encoding = 'utf-8'

# JSON 데이터 로드

CSAT_questions_json = response.json()

# pandas DataFrame으로 변환
df_check = pd.DataFrame(CSAT_questions_json)

# 출력
print(df_check.head())


                   id                                          paragraph  \
0    2023_11_KICE_1-3  사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 ...   
1    2023_11_KICE_4-9  (가)[A](중국에서 비롯된 유서(類書)는 고금의 서적에서 자료를 수집하고 항목별로...   
2  2023_11_KICE_10-13  법령의 조문은 대개 ‘A에 해당하면 B를 해야 한다.’처럼 요건과효과로 구성된 조건...   
3  2023_11_KICE_14-17  하루에 필요한 에너지의 양은 하루 동안의 총 열량 소모량인 대사량으로 구한다. 그중...   
4  2023_11_KICE_18-21  혼례를 마친 후 최척이 아내와 함께 장모를 모시고 집으로 돌아오매 하인들이 기뻐했다...   

   type                                           problems  
0     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
1     0  [{'question': '(가)와 (나)에 대한 설명으로 가장 적절한 것은?', ...  
2     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
3     0  [{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...  
4     1  [{'question': '윗글에 대한 설명으로 가장 적절한 것은?', 'choic...  


## [MYCODE] problems 항목 평준화

In [ ]:
def create_flattend_data(data):
  flattened_data = []
  for entry in data:
     problems = entry['problems']
     for problem in problems:
      flattened_problem = {
          'id': entry['id'],
          'paragraph': entry['paragraph'],
          'question': problem['question'],
          'question_plus': problem.get('question_plus', None),  # 'question_plus'가 없을 수도 있기 때문에 기본값 None 설정
          'choices': problem['choices'],
          'answer': problem['answer'],
          'score': problem['score']
                }
      flattened_data.append(flattened_problem)
  return pd.DataFrame(flattened_data)

# 결과 확인
#print(create_flattend_data(CSAT_questions_json))

## [MYCODE] 수능 국어 문제 생성



In [ ]:
def generate_batch_prompt(df, batch_size):
    prompts = []  # 배치별로 생성된 prompt들을 저장할 리스트

    # DataFrame의 각 row를 순회
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i + batch_size]

        prompt = "다음 문단에서 각 질문의 정답을 **###숫자** 형식으로 작성하세요. 모르는내용은 ###0으로 표기해주세요 예시: ###1, ###2, ###3.\n\n"

        # 각 배치에 대해 문단, 질문 및 선택지를 추가
        for idx, row in batch_df.iterrows():
            prompt += f"{idx+1}번 문단: {row['paragraph']}\n"  # 문단
            if row['question_plus']:
                prompt += f"{idx+1}번 보기: {row['question_plus']}\n"  # 보기
            prompt += f"{idx+1}번 선택지: \n"
            for choice_idx, choice in enumerate(row['choices'], start=1):
                prompt += f"{choice_idx}. {choice}\n"  # 선택지 번호 붙이기
            prompt += f"{idx+1}번 질문: {row['question']}"  # 질문
            if row['question_plus']:
              prompt += f"({idx+1}번 문단을 분석한 후 {idx+1}번 보기를 참고하여 {idx+1}번 질문에 답을 {idx+1}번 선택지에서 고르세요).\n\n"
            else:
              prompt += f"({idx+1}번 문단을 분석한 후 {idx+1}번 선택지에서 {idx+1}번 질문의 답을 고르세요).\n\n"

        # 최종 프롬프트를 리스트에 추가
        prompts.append(prompt)

    return prompts

In [ ]:
len(generate_batch_prompt(create_flattend_data(CSAT_questions_json),3))

15

In [ ]:
batch_prompt = generate_batch_prompt(create_flattend_data(CSAT_questions_json),3)
print(batch_prompt[4])

다음 문단에서 각 질문의 정답을 **###숫자** 형식으로 작성하세요. 모르는내용은 ###0으로 표기해주세요 예시: ###1, ###2, ###3.

13번 문단: 법령의 조문은 대개 ‘A에 해당하면 B를 해야 한다.’처럼 요건과효과로 구성된 조건문으로 규정된다. 하지만 그 요건이나 효과가항상 일의적인 것은 아니다. 법조문에는 구체적 상황을 고려해야그 상황에 ⓐ(맞는) 진정한 의미가 파악되는 불확정 개념이 사용될 수 있기 때문이다. 개인 간 법률관계를 규율하는 민법에서 불확정 개념이 사용된 예로 ‘손해 배상 예정액이 부당히 과다한경우에는 법원은 적당히 감액할 수 있다.’라는 조문을 ⓑ(들) 수 있다. 이때 법원은 요건과 효과를 재량으로 판단할 수 있다. 손해배상 예정액은 위약금의 일종이며, 계약 위반에 대한 제재인 위약벌도 위약금에 속한다. 위약금의 성격이 둘 중 무엇인지 증명되지 못하면 손해 배상 예정액으로 다루어진다.채무자의 잘못으로 계약 내용이 실현되지 못하여 계약 위반이발생하면, 이로 인해 손해를 입은 채권자가 손해 액수를 증명해야 그 액수만큼 손해 배상금을 받을 수 있다. 그러나 손해 배상 예정액이 정해져 있었다면 채권자는 손해 액수를 증명하지 않아도 손해 배상 예정액만큼 손해 배상금을 받을 수 있다. 이때 손해 액수가 얼마로 증명되든 손해 배상 예정액보다 더 받을 수는 없다. 한편 위약금이 위약벌임이 증명되면 채권자는 위약벌에 해당하는 위약금을 ⓒ(받을) 수 있고, 손해 배상 예정액과는 달리 법원이 감액할 수 없다. 이때 채권자가 손해 액수를증명하면 손해 배상금도 받을 수 있다.불확정 개념은 행정 법령에도 사용된다. 행정 법령은 행정청이구체적 사실에 대해 행하는 법 집행인 행정 작용을 규율한다. 법령상 요건이 충족되면 그 효과로서 행정청이 반드시 해야 하는특정 내용의 행정 작용은 기속 행위이다. 반면 법령상 요건이 충족되더라도 그 효과인 행정 작용의 구체적 내용을 ⓓ(고를)수 있는 재량이 행정청에 주어져 있을 때, 이러한 재량을 행사하는 행정 작용은

## [MYCODE] GPT prediction def 정의


In [ ]:
import re
def prediction(prompt,flattend_df):

    temperature = 0  # 창의성 및 각 token을 샘플링할 때 사용하는 temperature 값입니다.
    max_tokens = 300  # 생성하는 최대 token 개수 입니다.
    n = 1  # 같은 질의에 대해 몇 개의 답변을 출력할지 결정합니다.
    frequency_penalty = 0.0  # 같은 단어가 반복적으로 나오는 것을 방지하기 위한 옵션입니다
    final_score = 0 # 최종 점수

    # GPT-4에게 질문을 던지고 예측을 받는 부분
    messages = [
    {"role": "system", "content": "모든 답변은 반드시 '###숫자' 형식으로만 작성되어야 합니다."},
    {"role": "system", "content": "각 질문에 대한 답변은 반드시 하나의 답으로만 답해야 합니다."},
    {"role": "system", "content": "당신은 수능 국어를 가르치는 선생님입니다."},
    {"role": "system", "content": "이 문제는 국어 평가원 기출 문제와 유사한 유형입니다. 지문과 선택지를 꼼꼼히 비교한 뒤, 가장 적절한 답을 선택하세요."},
    {"role": "system", "content": "지문의 중심 내용과 주제를 파악해서 질문의 답하세요."},
    {"role": "system", "content": "선택지의 표현이 지문과 다르거나 과장된 경우를 주의하세요."},
    {"role": "system", "content": "선택지를 검토하며 가장 가능성이 높은 정답부터 우선 평가하세요."},
    {"role": "user", "content": prompt}
]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        n=n,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty
    )

    #print(f"response : {response}")

    # GPT-4의 답변 추출
    gpt_answer = response.choices[0].message.content
    print(f"gpt_answer : {response.choices[0].message.content}")
    # "정답: " 부분을 제거하고, 숫자만 추출하여 리스트로 변환
    # 정규식을 사용하여 ### 뒤의 숫자 추출
    gpt_answer_list = re.findall(r"###(\d+)", gpt_answer)
    print(f"gpt_answer_list : {gpt_answer_list}")

    return gpt_answer_list

## [MYCODE] 수능 국어 샘플로 문제 풀어보기

In [ ]:
# 문제 점수 계산
def score_check(response,df):
  answer_score_pairs = list(zip(df['answer'], df['score']))
  final_score = 0
  for idx, (answer, score) in enumerate(answer_score_pairs[:len(response)]):
    print(f"answer: {answer}, response[idx]: {response[idx]} , score : {score}")
    if str(answer) == response[idx]:
      final_score += score
  return final_score

In [ ]:
# 데이터 평준화
flattend_df = create_flattend_data(CSAT_questions_json)


# 샘플 예제 예측 및 결과
gpt_answer_list = prediction(batch_prompt[8],flattend_df)

# 스코어 계산
score = score_check(gpt_answer_list, flattend_df)
print(f"샘플 데이터로 준 결과 {score}점 입니다.")


gpt_answer : ###3, ###4, ###1
gpt_answer_list : ['3', '4', '1']
answer: 4, response[idx]: 3 , score : 2
answer: 5, response[idx]: 4 , score : 3
answer: 1, response[idx]: 1 , score : 2
샘플 데이터로 준 결과 2점 입니다.


## [MYCODE] 최종 결과

In [ ]:
# 수능 국어 문제를 풀어보기

# 전체 문제fmf batch 별로 나눠서 생성  prompt 작성
batch_size = 3
batch_prompts = generate_batch_prompt(flattend_df, batch_size)

# 전체 문제 예측 및 최종 결과
final_gpt_answer_list = [] # 결과 저장
for batch_prompt in batch_prompts:
  final_gpt_answer_list.append(prediction(batch_prompt, flattend_df))

print(final_gpt_answer_list)

gpt_answer : ###4, ###1, ###1
gpt_answer_list : ['4', '1', '1']
gpt_answer : ###4, ###5, ###3
gpt_answer_list : ['4', '5', '3']
gpt_answer : ###2, ###5, ###2
gpt_answer_list : ['2', '5', '2']
gpt_answer : ###4, ###5, ###2
gpt_answer_list : ['4', '5', '2']
gpt_answer : ###4, ###3, ###3
gpt_answer_list : ['4', '3', '3']
gpt_answer : ###5, ###1, ###4
gpt_answer_list : ['5', '1', '4']
gpt_answer : ###3, ###3, ###5
gpt_answer_list : ['3', '3', '5']
gpt_answer : ###3, ###5, ###3
gpt_answer_list : ['3', '5', '3']
gpt_answer : ###3, ###4, ###1
gpt_answer_list : ['3', '4', '1']
gpt_answer : ###5, ###1, ###2
gpt_answer_list : ['5', '1', '2']
gpt_answer : ###1, ###4, ###2
gpt_answer_list : ['1', '4', '2']
gpt_answer : ###3, ###1, ###5
gpt_answer_list : ['3', '1', '5']
gpt_answer : ###3, ###4, ###2
gpt_answer_list : ['3', '4', '2']
gpt_answer : ###5, ###2, ###4
gpt_answer_list : ['5', '2', '4']
gpt_answer : ###5, ###1, ###5
gpt_answer_list : ['5', '1', '5']
[['4', '1', '1'], ['4', '5', '3'], ['2',

In [ ]:
## 최종 점수
# 평탄화
flattened_gpt_answer_list = [item for sublist in final_gpt_answer_list for item in sublist]

final_score = score_check(flattened_gpt_answer_list, flattend_df)
print(f"GPT 님의 점수는 {final_score} 입니다.")

answer: 4, response[idx]: 4 , score : 2
answer: 5, response[idx]: 1 , score : 3
answer: 1, response[idx]: 1 , score : 2
answer: 4, response[idx]: 4 , score : 2
answer: 5, response[idx]: 5 , score : 2
answer: 3, response[idx]: 3 , score : 2
answer: 2, response[idx]: 2 , score : 2
answer: 5, response[idx]: 5 , score : 3
answer: 2, response[idx]: 2 , score : 2
answer: 4, response[idx]: 4 , score : 2
answer: 5, response[idx]: 5 , score : 2
answer: 2, response[idx]: 2 , score : 3
answer: 5, response[idx]: 4 , score : 2
answer: 3, response[idx]: 3 , score : 2
answer: 4, response[idx]: 3 , score : 2
answer: 4, response[idx]: 5 , score : 2
answer: 1, response[idx]: 1 , score : 3
answer: 4, response[idx]: 4 , score : 2
answer: 3, response[idx]: 3 , score : 2
answer: 3, response[idx]: 3 , score : 2
answer: 5, response[idx]: 5 , score : 3
answer: 1, response[idx]: 3 , score : 2
answer: 5, response[idx]: 5 , score : 2
answer: 3, response[idx]: 3 , score : 3
answer: 3, response[idx]: 3 , score : 2


In [ ]:
# 전체 문제fmf batch 별로 나눠서 생성  prompt 작성
batch_size = 1
batch_prompts = generate_batch_prompt(flattend_df, batch_size)

# 전체 문제 예측 및 최종 결과
final_gpt_answer_list_onliyone = [] # 결과 저장
for batch_prompt in batch_prompts:
  final_gpt_answer_list_onliyone.append(prediction(batch_prompt, flattend_df))

print(final_gpt_answer_list_onliyone)

gpt_answer : ###4
gpt_answer_list : ['4']
gpt_answer : ###5
gpt_answer_list : ['5']
gpt_answer : ###1
gpt_answer_list : ['1']
gpt_answer : ###4
gpt_answer_list : ['4']
gpt_answer : ###5
gpt_answer_list : ['5']
gpt_answer : ###3
gpt_answer_list : ['3']
gpt_answer : ###2
gpt_answer_list : ['2']
gpt_answer : ###5
gpt_answer_list : ['5']
gpt_answer : ###2
gpt_answer_list : ['2']
gpt_answer : ###4
gpt_answer_list : ['4']
gpt_answer : ###5
gpt_answer_list : ['5']
gpt_answer : ###2
gpt_answer_list : ['2']
gpt_answer : ###5
gpt_answer_list : ['5']
gpt_answer : ###3
gpt_answer_list : ['3']
gpt_answer : ###5
gpt_answer_list : ['5']
gpt_answer : ###4
gpt_answer_list : ['4']
gpt_answer : ###1
gpt_answer_list : ['1']
gpt_answer : ###4
gpt_answer_list : ['4']
gpt_answer : ###3
gpt_answer_list : ['3']
gpt_answer : ###3
gpt_answer_list : ['3']
gpt_answer : ###5
gpt_answer_list : ['5']
gpt_answer : ###1
gpt_answer_list : ['1']
gpt_answer : ###5
gpt_answer_list : ['5']
gpt_answer : ###3
gpt_answer_list 

In [ ]:
## 최종 점수
# 평탄화
flattened_gpt_one_answer_list = [item for sublist in final_gpt_answer_list_onliyone for item in sublist]

final_score = score_check(flattened_gpt_one_answer_list, flattend_df)
print(f"GPT 님의 점수는 {final_score} 입니다.")

answer: 4, response[idx]: 4 , score : 2
answer: 5, response[idx]: 5 , score : 3
answer: 1, response[idx]: 1 , score : 2
answer: 4, response[idx]: 4 , score : 2
answer: 5, response[idx]: 5 , score : 2
answer: 3, response[idx]: 3 , score : 2
answer: 2, response[idx]: 2 , score : 2
answer: 5, response[idx]: 5 , score : 3
answer: 2, response[idx]: 2 , score : 2
answer: 4, response[idx]: 4 , score : 2
answer: 5, response[idx]: 5 , score : 2
answer: 2, response[idx]: 2 , score : 3
answer: 5, response[idx]: 5 , score : 2
answer: 3, response[idx]: 3 , score : 2
answer: 4, response[idx]: 5 , score : 2
answer: 4, response[idx]: 4 , score : 2
answer: 1, response[idx]: 1 , score : 3
answer: 4, response[idx]: 4 , score : 2
answer: 3, response[idx]: 3 , score : 2
answer: 3, response[idx]: 3 , score : 2
answer: 5, response[idx]: 5 , score : 3
answer: 1, response[idx]: 1 , score : 2
answer: 5, response[idx]: 5 , score : 2
answer: 3, response[idx]: 3 , score : 3
answer: 3, response[idx]: 3 , score : 2


##[FEEDBAK] 문제를 45개의 문제를 3개씩 쪼개어 실행했을때는 80점까지 나오지만 1개의 문제로만 진행했을때는 정답률이 94점까지 나온걸 보면 높아지는 걸 확인했습니다. 여러개의 문제를 한번에 줄때 어떤 부분을 보충해서 해보면 정답률을 높일 수 있나요?